In [1]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os
from multiprocessing import Process, Queue
import random
import math
import time

In [ ]:
class Agent:
    def __init__(self, numberOfCurrencies, timeFrame, sess, initialPortfolio=10000.0):
        self._s = sess
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.conv1 = tf.layers.conv2d(inputs=self.inputT, filters=30, kernel_size=[1,3], activation=tf.nn.leaky_relu)
      #  self.conv1 = tf.nn.depthwise_conv2d(self.inputT, [1,3,3,4], [1,1,1,1], 'SAME')
        #self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=150, kernel_size=[1,8], activation=tf.nn.relu)
        #self.conv3 = tf.layers.conv2d(inputs = self.conv2, filters=200, kernel_size=[1,41], activation=tf.nn.relu)
        #self.conv4 = tf.layers.conv2d(inputs=self.conv3, filters=30, kernel_size=[1,1] , activation=tf.nn.relu)
        
        self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=30, kernel_size=[1,48], activation=tf.nn.leaky_relu)
        
        
        self.hidden = tf.layers.dense(self.conv2, 2000, activation=tf.nn.leaky_relu)
        #self.drop = tf.layers.dropout(self.hidden)
        self.final2 = tf.layers.dense(self.hidden, 1)
        self._allocate = tf.nn.softmax(self.final2, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        #self.loss = -tf.matmul(tf.matrix_transpose(tf.nn.leaky_relu(tf.log(self.priceChanges), alpha=10)),tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))
        self.averageLoss = tf.reduce_mean(tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))

        self.loss = tf.exp(tf.reduce_sum(tf.multiply(-tf.log(self.priceChanges), tf.reshape(self._allocate, [tf.shape(self._allocate)[0], numberOfCurrencies, 1]))))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run(self.averageLoss, feed_dict={self.inputT : obs, self.priceChanges: prices}))
        print(prices)

In [ ]:
def main():
    feed = feedBackup.copy()
    seeds = [3, 5, 7]
    tf.reset_default_graph()
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    tf.set_random_seed(seeds[1])
    test1 = Agent(len(testSim.currencies), 50, sess)
    sess.run(tf.global_variables_initializer())

    testData = feed[0]
    feed.pop(0)

    #print(testData)


    try:
        for episode in range(500):
            print("Episode: {}".format(episode))
            index = list(range(len(feed)))
            loss = 0
            count = len(feed)
            while len(index) != 0:
                for i in random.sample(index, 1):
                    with tf.device('/gpu:0'):
                        loss += test1.train_step(feed[i][0], feed[i][1])
                    index.remove(i)
            print(loss/count)
    except KeyboardInterrupt:
        print("Ending")
        test1.test_model(testData[0], testData[1])
    finally:
        print("Ending")
        test1.test_model(testData[0], testData[1])
            
if __name__ == "__main__":
    main()

In [2]:
class DataManager:
    def __init__(self, simulator, batch_size, batch_count=0, period_size=50, batch_offset=0):
        self.batchPath = os.path.abspath("./Batches")
        self.labelPath = os.path.abspath("./Labels")
        self.threads = 24
        self.batch = batch_count
        self.batch_size = batch_size
        self.simulator = simulator
        self.period_size = period_size
        self.offset = batch_offset
        if batch_count == 0:
            minSize = math.inf
            for pair in self.simulator.df.values():
                if len(pair.index) < minSize:
                    minSize = len(pair.index)
            print(minSize)
            self.batch = int(math.floor(minSize / batch_size))
            
    def importData(self):
        testSim = self.simulator
        q = Queue()
        jobs = []
        PERIOD_SIZE = self.period_size
        BATCH_SIZE = self.batch_size
        #BATCH_COUNT = int(math.floor(minSize / BATCH_SIZE))
        BATCH_COUNT = self.batch
        BATCH_OFFSET = self.offset
        dates = testSim.getAllDates()
        index = list(range(BATCH_COUNT))
        feed = []

        running = False
        count = 0
        countFile = 0
        while 1:
            if count < self.threads and len(index) != 0:
                for i in random.sample(index, self.threads-count if len(index) >= self.threads-count else len(index)):
                    p = Process(target=testSim.processTimePeriod, args=(q, PERIOD_SIZE, dates, BATCH_SIZE * (i + BATCH_OFFSET) + PERIOD_SIZE, BATCH_SIZE))
                    jobs.append(p)
                    p.start()
                    index.remove(i)
                print("Total unstarted:{}".format(len(index)))
            count = 0
            for p in jobs:
                if not p.is_alive():
                    p.terminate()
                    jobs.remove(p)
                else:
                    count += 1
            while not q.empty():
#                feed.append(q.get())
                pair = q.get()
                np.save((os.path.join(self.batchPath, "Batch_" + str(countFile))), pair[0])
                np.save((os.path.join(self.labelPath, "Label_" + str(countFile))), pair[1])
                countFile += 1
            if count == 0 and len(index) == 0 and countFile == BATCH_COUNT:
                break
#         print(len(feed))
#         return feed
    
    def prepareData(self, reset=False):
        if not os.path.exists(self.batchPath):
            os.makedirs(self.batchPath)
        if not os.path.exists(self.labelPath):
            os.makedirs(self.labelPath)
        if reset:
            for folder in (self.batchPath, self.labelPath):
                for the_file in os.listdir(folder):
                    file_path = os.path.join(folder, the_file)
                    try:
                        if os.path.isfile(file_path):
                            os.unlink(file_path)
                    except:
                        pass
        if "Done" in os.listdir(self.batchPath):
            return
        now = time.time()
        fullData = self.importData()
        later = time.time()
        print('Total time for all data:{} seconds'.format(int(later-now)))
        count = 0
#         for pair in fullData:
#             np.save((os.path.join(self.batchPath, "Batch_" + str(count))), pair[0])
#             np.save((os.path.join(self.labelPath, "Label_" + str(count))), pair[1])
#             count += 1
        f = open(os.path.join(self.batchPath, "Done"), 'w')
        f.close()
        
    def readData(self):
        availableData = os.listdir(self.batchPath)
        if "Done" in availableData:
            availableData.remove("Done")
        indices = range(len(availableData))
        n_tasks = len(availableData)/self.threads
        print(n_tasks)
        q = Queue()
        if n_tasks == 0:
            return
        offSet = 0
        jobs = []
        feed = []
        for i in range(self.threads):
            p = Process(target=self.readFiles, args=(int(offSet), int(offSet+n_tasks), q))
            jobs.append(p)
            print("Starting")
            p.start() 
            offSet += n_tasks
        while not q.empty() or len(feed) != self.batch:
            feed.append(q.get())
        return feed
            
        
    def readFiles(self, fileRangeStart, fileRangeEnd, finalQ):
        for i in range(fileRangeStart, fileRangeEnd):
            finalQ.put((str(i), np.load(os.path.join(self.batchPath, "Batch_" + str(i) + ".npy")), np.load(os.path.join(self.labelPath, "Label_"+ str(i) + ".npy"))))

In [4]:
data = DataManager(testSim, 2000)

data.prepareData(reset=True)



958922
Total unstarted:455
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


Process Process-12:
Process Process-18:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 229, in processTimePeriod
    allRates[count] = (self.getRates(timeIndex+indexOffset))
  File "/Users/nabicanozberkman/Lea

  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 126, in getRates
    index = self.df[pair].loc[self.df[pair]['Timestamp'] == self.allDates[dateIndex + offSet]].index.values
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 126, in getRates
    index = self.df[pair].loc[self.df[pair]['Timestamp'] == self.allDates[dateIndex + offSet]].index.values
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1283, in wrapper
    res = na_op(values, other)
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 229, in processTimePeriod
    allRates[count] = (self.getRates(timeIndex+indexOffset))
  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1283, in wrapper
    res = na_op(values, other)
  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1283, in wrapper
    res = na_op(values, other)
  File "/usr/local/Cellar/python/3.6.5/Fram

  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1143, in na_op
    result = _comp_method_OBJECT_ARRAY(op, x, y)
  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1122, in _comp_method_OBJECT_ARRAY
    result = libops.scalar_compare(x, y, op)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1143, in na_op
    result = _comp_method_OBJECT_ARRAY(op, x, y)
Traceback (most recent call last):
KeyboardInterrupt
Process Process-16:
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 229, in processTimePeriod
    allRates[count] = (self.getRates(timeIndex+indexOffset))
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.p

KeyboardInterrupt: 

  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/site-packages/pandas/core/ops.py", line 1122, in _comp_method_OBJECT_ARRAY
    result = libops.scalar_compare(x, y, op)
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 126, in getRates
    index = self.df[pair].loc[self.df[pair]['Timestamp'] == self.allDates[dateIndex + offSet]].index.values
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
KeyboardInterrupt
KeyboardInterrupt
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py", line 126, in getRates
    index = self.df[pair].loc[self.df[pair]['Timestamp'] == self.allDates[dateIndex + offSet]].index.values
  File "/Users/nabicanozberkman/Learning/Trader/Agent/trade_sim.py

In [ ]:
print(np.array_equal(test2[0][2], test1[0][1]))


In [3]:
testSim = Market(['EUR','USD', 'TRY', 'JPY'], os.path.abspath('../Data_Processing/ProcessedData'))